# Projeto 1 - Ciência dos Dados

Nome: Vinícius de Barros Gomes

Nome: Gabriel Nakano Fujihara

Nome: Pedro Roberto Fernandes Noronha

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [170]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from unidecode import unidecode
from sklearn.model_selection import train_test_split

In [171]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\gabri\OneDrive - Insper - Institudo de Ensino e Pesquisa\Insper\2023\1-semestre\DSoft(desing de software)\DataScienceProject


Carregando a base de dados com os tweets classificados manualmente:

In [172]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)


,Mensagem,"Target (Loja e Sistema, Entrega,Conteudo)"
0,"Não li, até pq não consegui baixar. Vcs podem ...",Loja e Sistema
1,Não tem nada indicando que é em português de P...,Loja e Sistema
2,"Produto horrível, veio com várias páginas em b...",Entrega
3,O minimo é não ler esse livro,Conteúdo do Livro
4,Parece que o autor não leu o próprio livro!! 🙄,Conteúdo do Livro


In [173]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,"Target (Loja e Sistema, Entrega,Conteudo)"
0,"Apesar da narrativa prender o leitor, os argum...",Conteúdo do Livro
1,A formatação desse ebook atrapalha muito a lei...,Loja e Sistema
2,Não é o primeiro livro da Record que oxida em ...,Entrega
3,O hype que envolve esse livro é ENORME então e...,Conteúdo do Livro
4,"meu deus, essa escritora conseguiu estragar um...",Conteúdo do Livro


___
## Classificador automático



Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

Buscamos avaliar se os livros oferecidos pela amazon são ou não bons, portanto, foi ignorado quando haviam críticas exclusivamente ao serviço da amazon, como entrega e ou qualidade física do produto.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [174]:
def cleanup(text):
    text=text.lower()
    punctuation = '[´"!-.:?;$''<>]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def remover_acentos(texto):
    # Use unidecode para remover acentos e converter caracteres especiais para caracteres ASCII
    texto_sem_acentos = unidecode(texto)
    return texto_sem_acentos


In [175]:
#Limpa o df
def df0(df):
    df['Mensagem'] = df['Mensagem'].apply(cleanup)
    df['Mensagem'] = df['Mensagem'].apply(remover_acentos)
    novo_df = pd.concat([df[['Mensagem']], df[['Target (Loja e Sistema, Entrega,Conteudo)']]], axis=1)
    return novo_df
train=df0(train)
train['Mensagem']=train['Mensagem'].astype('category')

In [176]:
df_Loja_Sistema=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Loja e Sistema']
df_Entrega=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Entrega']
df_Conteudo=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Conteúdo do Livro']
quant_loja=len(df_Loja_Sistema)
quant_Conteudo=len(df_Conteudo)
quant_Entrega=len(df_Entrega)


In [177]:
#Conta a quantidade de palavras dentro de um dataframe
def ll(df):
    todas_as_frases=df["Mensagem"]
    todas_as_palavras = []
    for i in todas_as_frases:
        frase=[]
        x=i.split(" ")
        for u in x:
            if u not in frase:
                frase.append(u)
        for d in frase:
            todas_as_palavras.append(d)
    todas_as_palavrasSeries=pd.Series(todas_as_palavras)
    return todas_as_palavrasSeries.value_counts()
quant_palavras_Entrega= ll(df_Entrega)
quant_palavras_Conteudo= ll(df_Conteudo)
quant_palavras_Loja= ll(df_Loja_Sistema)

In [178]:
quantidade_total=quant_loja + quant_Conteudo + quant_Entrega
def laplace(x,q):
    return (x+1)/(q+quantidade_total)

In [179]:
frase=str(input('frase:'))
frase=cleanup(frase)
frase=remover_acentos(frase)
frase=frase.lower()
palavras=frase.split(' ')
prob_Loja_Sistema=1
prob_Entrega=1
prob_Conteudo=1
for palavra in palavras:
    if palavra in quant_palavras_Loja:
        prob_Loja_Sistema*=(quant_palavras_Loja[palavra]/quant_loja)
    else:
        prob_Loja_Sistema*=laplace(0,quant_loja)
for palavra in palavras:
    if palavra in quant_palavras_Entrega:
        prob_Entrega*=(quant_palavras_Entrega[palavra]/quant_Entrega)
    else:
        prob_Entrega*=laplace(0,quant_Entrega)
for palavra in palavras:
    if palavra in quant_palavras_Conteudo:
        prob_Conteudo*=(quant_palavras_Conteudo[palavra]/quant_Conteudo)
    else:
        prob_Conteudo*=laplace(0,quant_Conteudo)
print(prob_Conteudo)
print(prob_Entrega)
print(prob_Loja_Sistema)
maior_prob=max(prob_Conteudo , prob_Entrega , prob_Loja_Sistema)
if maior_prob==prob_Conteudo:
    print("Conteúdo do Livro")
elif maior_prob==prob_Entrega:
    print("Entrega")
elif maior_prob==prob_Loja_Sistema:
    print("Loja e Sistema")

0.0008453085376162299
0.001184834123222749
0.0011668611435239206
Entrega


In [180]:
#Classificando o dataframe teste com o classificador
def classifica(df):
    df_Loja_Sistema=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Loja e Sistema']
    df_Entrega=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Entrega']
    df_Conteudo=train.loc[train["Target (Loja e Sistema, Entrega,Conteudo)"]=='Conteúdo do Livro']
    quant_loja=len(df_Loja_Sistema)
    quant_Conteudo=len(df_Conteudo)
    quant_Entrega=len(df_Entrega)
    quant_palavras_Entrega= ll(df_Entrega)
    quant_palavras_Conteudo= ll(df_Conteudo)
    quant_palavras_Loja= ll(df_Loja_Sistema)
    df['Mensagem']=df['Mensagem'].astype('category')
    coluna_de_resposta=[]
    for frase in df["Mensagem"]:
        frase=cleanup(frase)
        frase=remover_acentos(frase)
        frase=frase.lower()
        palavras=frase.split(' ')
        prob_Loja_Sistema=1
        prob_Entrega=1
        prob_Conteudo=1
        for palavra in palavras:
            if palavra in quant_palavras_Loja:
                prob_Loja_Sistema*=(quant_palavras_Loja[palavra]/quant_loja)
            else:
                prob_Loja_Sistema*=laplace(0,quant_loja)
        for palavra in palavras:
            if palavra in quant_palavras_Entrega:
                prob_Entrega*=(quant_palavras_Entrega[palavra]/quant_Entrega)
            else:
                prob_Entrega*=laplace(0,quant_Entrega)
        for palavra in palavras:
            if palavra in quant_palavras_Conteudo:
                prob_Conteudo*=(quant_palavras_Conteudo[palavra]/quant_Conteudo)
            else:
                prob_Conteudo*=laplace(0,quant_Conteudo)
        maior_prob=max(prob_Conteudo , prob_Entrega , prob_Loja_Sistema)
        if maior_prob==prob_Conteudo:
            coluna_de_resposta.append('Conteúdo do Livro')
        elif maior_prob==prob_Entrega:
            coluna_de_resposta.append("Entrega")
        elif maior_prob==prob_Loja_Sistema:
            coluna_de_resposta.append("Loja e Sistema")
    coluna_de_resposta_Series=pd.Series(coluna_de_resposta)
    df['Respostas']=coluna_de_resposta_Series
    novo_df = pd.concat([df['Mensagem'],df['Target (Loja e Sistema, Entrega,Conteudo)'],df['Respostas']], axis=1)
    return novo_df

novo_classificado=classifica(test)


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [181]:
#Obtem as classificações certas e erradas da função classificador
resposta_do_classifica = novo_classificado 
resposta_do_classifica  =resposta_do_classifica['Respostas']
resposta_real = test['Target (Loja e Sistema, Entrega,Conteudo)']

acerto_conteudo = 0
acerto_entrega = 0
acerto_loja  = 0
numero_de_conteudo = 0
numero_de_entrega = 0
numero_de_loja = 0

for x in range( len (resposta_real)):
    if resposta_real.iloc[x] == 'Entrega' :
        numero_de_entrega += 1
        if resposta_do_classifica.iloc[x] == resposta_real.iloc[x]:
            acerto_entrega += 1
    elif resposta_real.iloc[x] == 'Loja e Sistema':
        numero_de_loja += 1
        if resposta_do_classifica.iloc[x] == resposta_real.iloc[x]:
            acerto_loja += 1
    elif resposta_real.iloc[x] == 'Conteúdo do Livro':
        numero_de_conteudo += 1
        if resposta_do_classifica.iloc[x] == resposta_real.iloc[x]:
            acerto_conteudo += 1

In [182]:
#calcula as probabilidades das classificações estarem erradas
probabilidade_verdadeiro_Entrega = acerto_entrega / numero_de_entrega
probabilidade_verdadeiro_loja=acerto_loja/numero_de_loja
probabilidade_verdadeiro_conteudo=acerto_conteudo/numero_de_conteudo
probabilidade_falso_Entrega=(numero_de_entrega-acerto_entrega)/numero_de_entrega
probabilidade_falso_loja=(numero_de_loja-acerto_loja)/numero_de_loja
probabilidade_falso_conteudo=(numero_de_conteudo-acerto_conteudo)/numero_de_conteudo

print('A probabilidade de ser uma mensagem de Entrega verdadeira é de {0:.2F}%'.format(probabilidade_verdadeiro_Entrega*100))
print('A probabilidade de ser uma mensagem de Conteúdo do Livro verdadeira é de {0:.2F}%'.format(probabilidade_verdadeiro_conteudo*100))
print('A probabilidade de ser uma mensagem de Loja e Sistema verdadeira é de {0:.2F}%'.format(probabilidade_verdadeiro_loja*100))
print('A probabilidade de não ser uma mensagem de Entrega verdadeira é de {0:.2F}%'.format(probabilidade_falso_Entrega*100))
print('A probabilidade de não ser uma mensagem de Conteúdo do Livro verdadeira é de {0:.2F}%'.format(probabilidade_falso_conteudo*100))
print('A probabilidade de não ser uma mensagem de Loja e Sistema verdadeira é de {0:.2F}%'.format(probabilidade_falso_loja*100))

A probabilidade de ser uma mensagem de Entrega verdadeira é de 86.67%
A probabilidade de ser uma mensagem de Conteúdo do Livro verdadeira é de 95.86%
A probabilidade de ser uma mensagem de Loja e Sistema verdadeira é de 56.86%
A probabilidade de não ser uma mensagem de Entrega verdadeira é de 13.33%
A probabilidade de não ser uma mensagem de Conteúdo do Livro verdadeira é de 4.14%
A probabilidade de não ser uma mensagem de Loja e Sistema verdadeira é de 43.14%


In [183]:
#verificando a acurácia do classificador a partir do arquivo teste
def acuracia(df):
    df_respostas=classifica(df)
    novo_df=pd.concat([df["Target (Loja e Sistema, Entrega,Conteudo)"],df_respostas['Respostas']], axis=1)
    acertos=0
    for i in novo_df.index:
        if novo_df["Target (Loja e Sistema, Entrega,Conteudo)"][i]==novo_df["Respostas"][i]:
            acertos+=1
    quantidade_respostas=len(df_respostas["Respostas"])
    return acertos*100/quantidade_respostas
print("Acurácia é:{0}%".format(acuracia(novo_classificado)))

Acurácia é:86.8%


___
### Concluindo

Nota-se que o classificador tem como forte a classificação das mensagens que tratam sobre o Conteúdo do Livro, também conseguindo classificar as mensagens de entrega. O problema do classificador são as mensagens relacionadas à Loja e Sistema (56.86%), isso se dá devido a diversas dessas mensagens também conterem comentários sobre o Conteúdo do Livro. Em geral o classificador consegue lidar com duplas negações e sarcasmos, já que as classificações em si não são atrapalhadas por eles. Se dado mais tempo e amostras o classificador pode ser melhorado e obter uma acuracia geral melhorada, além ter a porcentagem de acerto das classificações relacionadas à Loja e Sistema incrementadas.

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

In [186]:
# Combine os DataFrames de treinamento e teste em um único DataFrame
def embaralha(train,test):
    combined_data = pd.concat([train, test], ignore_index=True)

    # Separe aleatoriamente 300 mensagens para treinamento e 200 para teste
    train, test = train_test_split(combined_data, test_size=200, train_size=300, random_state=42)
    return train, test

In [189]:
acurracy_list=[]
for i in range(100):
    x=embaralha(train,test)
    y=x[0]
    z=x[1]
    p=classifica(z)
    p=acuracia(p)
    acurracy_list.append(p)
print(acurracy_list)


[7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5, 7.5]


___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**